### Extract functions from LL files

In [1]:
import CodePreprocessing as preprocessing
import json
import re

In [2]:

# ----------------- user functions  --------------------
preprocessing.functions_preprocessing( llvm_file='pairs/UserCode/UserCode.ll', json_file='UserCode' )
with open( 'UserCode.json', 'r' ) as f:
    user_code = json.load( f ) 

# ----------------- Vulnerable function ------------------
preprocessing.functions_preprocessing(llvm_file='pairs/ourVulnCodes/VulnerableCode.ll', json_file='VulnerableCode' )
with open( 'VulnerableCode.json', 'r' ) as f:
    vulnerable_code = json.load( f )

vulnerable_function=[]
for key in vulnerable_code:
    vulnerable_function.append(vulnerable_code[key])


### Matching

In [3]:
import matchers as matcher
import LLNormalizer as normalizer
import Winnowing

In [4]:
def check_function_vulnerable(threshold,score1,score2,score3):
    return score1 > threshold and score2 > threshold and score3 > threshold

In [5]:
#applying KNN but with similarity measures, we take the top 3 scores in all similarity measures, if those top 3 passed the threshold, we do the ultimate test, MOSS.
#K here equals 3

threshold=0.7
k=3

#this is a dictionary of key: vulnerable function (which is in our database)        value: code_scores for this vulnerable function
Vulnerable_Matches = dict()

for k,v in vulnerable_code.items():
    vuln_func = v
    vuln_head=k
    code_scores=dict()


    for key in user_code:
        fn=user_code[key] 
        #fn=normalizer.NormalizeLLVM(fn)

        jaro_winkler=matcher.jaro_winkler_similarity(fn, vuln_func)
        levenshtein=matcher.levenshtein_similarity(fn, vuln_func)
        ratcliff_obershelp=matcher.ratcliff_obershelp_similarity(fn, vuln_func)
        trigram=matcher.trigram_similarity(fn, vuln_func)
        sorensen_dice=matcher.sorensen_dice_similarity(fn, vuln_func)
        jaccard_distance=matcher.jaccard_distance(fn, vuln_func)

        scores=[jaro_winkler,levenshtein,ratcliff_obershelp,trigram,sorensen_dice,jaccard_distance]
        scores.sort(reverse=True) 
        code_scores[key]=scores
    
    Vulnerable_Matches[vuln_head] = code_scores

with open('code_scores.json', 'w') as f:
    f.write(json.dumps(Vulnerable_Matches, indent=6))

# MOSS

In [6]:
Candidate_Functions = []
for k,v in vulnerable_code.items():
    vuln_func = v
    vuln_head = k

    #Normalizing the vulnerable function with us (not user code)
    normalizedvuln = normalizer.NormalizeLLVM(vuln_func)

    #for each function passing the threshold, do MOSS.
    for key in Vulnerable_Matches[vuln_head]:
        code_scores = Vulnerable_Matches[vuln_head]
        if check_function_vulnerable(threshold,code_scores[key][0],code_scores[key][1],code_scores[key][2]):
            fn = user_code[key]

            #sometimes normalizing behaves good, sometimes bad.
            normalizedfn = normalizer.NormalizeLLVM(fn)
            
            #MOSS Metrics (defined in Winnowing.py), Parameters passed: k=20, ws = 10, P=10
            MOSS_Acc_metric1, MOSS_Acc_metric2, hits, misses1, misses2 = Winnowing.diff(normalizedfn, normalizedvuln, K= 20, WindowSize= 10, P= 10)
            
            print("\n\nVulnerable function found:",key)
            Candidate_Functions.append(re.findall('(@.*)\(', key)[0])

            #MOSS Thresholds, 0.7 for Metric1, 0.7 for Metric2, those thresholds are highly dependent on the vulnerability type unfortunately.
            if(MOSS_Acc_metric1>0.7 or MOSS_Acc_metric2>0.7):
                print(f"MOSS Caught this <3 !!!")
                print(f"Accuracy_Metric 1 = {MOSS_Acc_metric1}  ||  Accuracy_Metric 2 = {MOSS_Acc_metric2}\nhits: {hits} , misses: {misses1}, misses2: {misses2}")
            print('-----------------------')



Vulnerable function found: define void @"CWE23_Relative_Path_Traversal__char_environment_fopen_41::bad"() local_unnamed_addr {

MOSS Caught this <3 !!!
Accuracy_Metric 1 = 0.9059233449477352  ||  Accuracy_Metric 2 = 0.5241935483870968
hits: 4420 , misses: 459, misses2: 4012
-----------------------


Vulnerable function found: define void @"CWE23_Relative_Path_Traversal__char_environment_fopen_41::goodG2B"() local_unnamed_addr {

MOSS Caught this <3 !!!
Accuracy_Metric 1 = 0.8165194164783234  ||  Accuracy_Metric 2 = 0.5601917113053284
hits: 3974 , misses: 893, misses2: 3120
-----------------------


# Graph matching ان شاء الله

In [8]:
import graph
import os
import re
from subprocess import run
import pathlib
import json

#Path to Marim's script generate_subgraphs.py
absPathtoCFGScript = str(os.path.abspath("../../IrToCFGs/generate_subgraphs.py")).replace("\\", "/")
absPathtoCFGScript = list(absPathtoCFGScript)
absPathtoCFGScript[0] = absPathtoCFGScript[0].upper()
absPathtoCFGScript = ''.join(absPathtoCFGScript)


#Path to the pairs folder in this directory
absPathtoPairsFolder = (str(os.getcwd())+"/pairs").replace("\\", "/")
absPathtoPairsFolder = list(absPathtoPairsFolder)
absPathtoPairsFolder[0] = absPathtoPairsFolder[0].upper()
absPathtoPairsFolder = ''.join(absPathtoPairsFolder)

#run CFG script on all subfolders inside pairs folder
run(["python",absPathtoCFGScript, "0", absPathtoPairsFolder , absPathtoPairsFolder])



CompletedProcess(args=['python', 'D:/ClassWork/Guardista/IrToCFGs/generate_subgraphs.py', '0', 'D:/ClassWork/Guardista/Localizer/Common/pairs', 'D:/ClassWork/Guardista/Localizer/Common/pairs'], returncode=0)

In [10]:
#Prepare graphs for Vulnerable code, precompute them and store them in a list
#Vulnerable code (we are storing) is put inside a folder called ourVulnCodes, and the corresponding CFGs is inside a folder called ourVulnCodes_subgraphs/VulnerableCode_subgraphs
VulnerableCodeSubgraphsFolder = absPathtoPairsFolder + "/ourVulnCodes_subgraphs/VulnerableCode_subgraphs"


allFiles = os.listdir(VulnerableCodeSubgraphsFolder)

#List containing graphs of each precomputed Vulnerable Code
ourGraphs = []
for jsonfile in allFiles:
    fulljsonFilePath = VulnerableCodeSubgraphsFolder+'/'+jsonfile
    if(pathlib.Path(jsonfile).suffix != ".json"):
        continue
    with open(fulljsonFilePath) as f:
        jsonDict= json.load(f)
    
    
    functionName = jsonDict["function_name"].replace('\\', '')
    newGraph = graph.Graph()
    newGraph.readGraphFromJSON(fulljsonFilePath)
    ourGraphs.append(newGraph)
    

In [11]:
'''
we are trying to find the json file containing the name of the candidate functions (we will not compute CFG of EVERY possible function, rather just the candidate functions),
if we found a json of a candidate function, we compute its graph and perform the matching.

candidate functions are the functions that passed MOSS
'''


UserCodeSubgraphsFolder = absPathtoPairsFolder + "/UserCode_subgraphs/UserCode_subgraphs"

final_Matched_Functions = []

allFiles = os.listdir(UserCodeSubgraphsFolder)
for jsonfile in allFiles:
    fulljsonFilePath = UserCodeSubgraphsFolder+'/'+jsonfile
    if(pathlib.Path(jsonfile).suffix != ".json"):
        continue
    with open(fulljsonFilePath) as f:
        jsonDict = json.load(f)
    functionName = jsonDict["function_name"].replace('\\', '')

    if(functionName in Candidate_Functions):
        #Construct User Graph and Perform Matching
        UserCodeGraph = graph.Graph()
        UserCodeGraph.readGraphFromJSON(fulljsonFilePath)
        MatchPairs = graph.matchGraphWithListOfGraphs(UserCodeGraph , ourGraphs, otherWayAround=True)
        
        #MatchPairs is a list of Tuples, each Tuples contains the UserFunction Name and the Function Name stored in our Database
        if(MatchPairs):
            final_Matched_Functions.append(MatchPairs)

# Final Matches الحمد لله

In [12]:
print(final_Matched_Functions)
# NOTICE NO FALSE POSITIVES NOR FALSE NEGATIVES <3 <3 <3

[[('@"CWE23_Relative_Path_Traversal__char_environment_fopen_41::bad"', '@"CWE23_Relative_Path_Traversal__char_environment_fopen_18::bad"'), ('@"CWE23_Relative_Path_Traversal__char_environment_fopen_41::bad"', '@"CWE23_Relative_Path_Traversal__char_environment_fopen_18::bad"')]]


In [13]:

#Cleanup

import os
files = os.listdir('./')
files = [ fi for fi in files if fi.endswith(".json")]
for f in files:
    os.remove(f)


PermissionError: [WinError 5] Access is denied: 'pairs'